# EVENT SAMPLER TUTORIAL

## Prerequisites 

....

### Introduction

This tutorial describes how to  simulate an observation of a sky field, starting from a given IRF and a Sky model. The output of this process is the creation of a .fits photon event list. 
The core of the simulation lies into the Gammapy `MapDatasetEventSampler` class. The class bases its capabilities on the inverse cumulative distribution function (https://en.wikipedia.org/wiki/Cumulative_distribution_function#Inverse_distribution_function_(quantile_function)). 

The `MapDatasetEventSampler` takes in input a `Dataset` object from which it evaluates a map of predicted counts per bin and it then samples the events giving true coordinates, true energies and times of the events. The class then can also apply IRF corrections (i.e. the PSF and the energy dispersion) in order to reconstruct the coordinates and the energies of the sampled events. 

The metadata in the event-list are then catched from an `Observation` object. 


### Setup

In [ ]:
from pathlib import Path
import numpy as np
import copy
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.data import DataStore, GTI, Observation
from gammapy.datasets import MapDataset, MapDatasetEventSampler
from gammapy.maps import MapAxis, WcsGeom, Map
from gammapy.irf import load_cta_irfs
from gammapy.makers import MapDatasetMaker
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    Model, Models, SkyModel, 
    PowerLawSpectralModel, 
    PointSpatialModel,
    GaussianSpatialModel,
    SkyDiffuseCube
)
from regions import CircleSkyRegion

### Define the Sky model
First of all, let's define a Sky model for a point-like source centered 0.5 deg far from the Galactic Center and with a power-law spectral shape. We then save the model into a yaml file.

In [ ]:
spectral_model_pwl = PowerLawSpectralModel(index=2,amplitude="1e-12 TeV-1 cm-2 s-1", reference='1 TeV')
spatial_model_point = PointSpatialModel(lon_0= "0 deg", lat_0="0.5 deg", frame='galactic')
sky_model_pntpwl = SkyModel(spectral_model=spectral_model_pwl, spatial_model=spatial_model_point)

models_pntpwl = Models([sky_model_pntpwl])

file_model = "point-pwl.yaml"
models_pntpwl.write(file_model, overwrite=True)

### Create the dataset
In this section we show how to create the dataset needed for the simulation (for more info about datasets, please visit https://docs.gammapy.org/dev/notebooks/analysis_2.html#Preparing-reduced-datasets-geometry). 

Hereafter, we select the IRF from the South configuration of the CTA DC1 to perform our observation simulations. We set the pointing position of the simulated field at the Galactic Center and we fix the exposure time to 8 hr. 

In [ ]:
IRF_FILE = "$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits"

POINTING = SkyCoord(0.0, 0.0, frame="galactic", unit="deg")
LIVETIME = 8 * u.hr

We then define the energy axis (true and reconstruncted), the migration axis and the geometry of the observation. 

We note that this is crucial point as the spatial and energetic binning needs to be very fine. In particular, we suggest to define the energies setting a minimum binning of least 10-20 bins per decade. Instead the spatial binning can be different from source to source and we suggest to adopt a binning significantly smaller than source size.


Here, we set the geometry of the dataset to a field of view of 4degx4deg, adopting a binsize of 0.02 deg which is enough for the examples that will be shown hereafter.

In [ ]:
# dataset config
ENERGY_AXIS = MapAxis.from_energy_bounds("0.1 TeV", "100 TeV", nbin=10, per_decade=True)
ENERGY_AXIS_TRUE = MapAxis.from_energy_bounds("0.03 TeV", "300 TeV", nbin=20, per_decade=True, name="energy_true")
MIGRA_AXIS = MapAxis.from_bounds(0.5, 2, nbin=150, node_type="edges", name="migra")

WCS_GEOM = WcsGeom.create(
    skydir=POINTING, width=(4, 4), binsz=0.02, frame="galactic", axes=[ENERGY_AXIS]
)

The generate dataset selecting from the IRF the background model, the psf and the edisp. We create an `Observation` object that containts the pointing position, the GTIs and the IRFs. The datase thus created can be saved into a FITS file simply the `write()` function.


In [ ]:
irfs = load_cta_irfs(IRF_FILE)
observation = Observation.create(
    obs_id=1001, pointing=POINTING, livetime=LIVETIME, irfs=irfs
)

empty = MapDataset.create(WCS_GEOM, energy_axis_true=ENERGY_AXIS_TRUE, migra_axis=MIGRA_AXIS)
maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])
dataset = maker.run(empty, observation)

dataset.write('dataset.fits.gz', overwrite=True)

### Simulate the source + background events

To simulate the Sky model, we need to provide the Sky model to the dataset:

In [ ]:
dataset.models.extend(models_pntpwl)
print(dataset.models)

We then create an `Observation` object, from which the event-sampler will take the meta-data to be written in the ouput event list...

In [ ]:
obs_id=1
observation = Observation.create(
    obs_id=obs_id, pointing=POINTING, livetime=LIVETIME, irfs=irfs
)

The next step shows how to sample the events with the `MapdatasetEventSampler` class. The class requests a random number seed generator that we set with `random_state=0`. 

In [ ]:
sampler = MapDatasetEventSampler(random_state=0)
events = sampler.run(dataset, observation)

The output of the event-sampler is an event list with coordinates, energies and time of arrivals of the source and background events. Source and background events are flagged by the MC_ID identifier (0 for the background and 2 for the source).

In [ ]:
events.table

In [ ]:
print(f"Source events: {len(np.where(events.table['MC_ID']==2)[0])}")
print(f"Background events: {len(np.where(events.table['MC_ID']==0)[0])}")

In [ ]:
events.table.write("events_0001.fits.gz", overwrite=True)

In [ ]:
events.peek()

### Generate a skymap
A skymap of the simulated events is the following:

In [ ]:
counts = Map.create(frame="galactic", skydir=(0, 0.), binsz=0.02, npix=(150, 150))
counts.fill_events(events)
counts.plot(add_cbar=True)

### Fit the simulated data
We can now check the sake of the event sampling by fitting the data (a tutorial of source fitting is here https://docs.gammapy.org/dev/notebooks/analysis_2.html). We make use of the same Sky model adopted for the simulation. 
Hence, we firstly read the dataset and the model file, and we fill the dataset with the events.

In [ ]:
dataset = MapDataset.read("dataset.fits.gz")
models_sim_point = Models.read("point-pwl.yaml")

counts = Map.from_geom(WCS_GEOM)
counts.fill_events(events)
dataset.counts = counts
dataset.models.extend(models_sim_point)

Let's fit the data and look at the results:

In [ ]:
fit = Fit([dataset])
result = fit.run(optimize_opts={"print_level": 1})
print(result)

In [ ]:
result.parameters.to_table()

The results are quite satisfactory and consistent with the simulated parameters within ~1 sigma.

### Extended source

In [ ]:
spatial_model_gauss = GaussianSpatialModel(lon_0= "0 deg", lat_0="0 deg", sigma="0.3 deg", frame='galactic')
sky_model_pntgaus = SkyModel(spectral_model=spectral_model_pwl, spatial_model=spatial_model_gauss)

models_pntgaus = Models([sky_model_pntgaus])

file_model = "gauss-pwl.yaml"
models_pntgaus.write(file_model, overwrite=True)

In [ ]:
dataset = MapDataset.read("dataset.fits.gz")
dataset.models.extend(models_pntgaus)
print(dataset.models)

In [ ]:
sampler = MapDatasetEventSampler(random_state=0)
events = sampler.run(dataset, observation)

In [ ]:
events.peek()

In [ ]:
counts = Map.create(frame="galactic", skydir=(0, 0.), binsz=0.02, npix=(150, 150))
counts.fill_events(events)
counts.smooth(0.04 * u.deg).plot(add_cbar=True)

In [ ]:
dataset = MapDataset.read("dataset.fits.gz")
models_sim_pntgaus = Models.read("gauss-pwl.yaml")

counts = Map.from_geom(WCS_GEOM)
counts.fill_events(events)
dataset.counts = counts
dataset.models.extend(models_sim_pntgaus)

In [ ]:
fit = Fit([dataset])
result = fit.run(optimize_opts={"print_level": 1})
print(result)

In [ ]:
result.parameters.to_table()

### Extended source using a template
Let's now see the case of the event sampling of a template map. Here we use the ... that it is possible to find here ...

In [ ]:
diffuse = SkyDiffuseCube.read(
    "$GAMMAPY_DATA/fermi-3fhl-gc/gll_iem_v06_gc.fits.gz"
)
models_diffuse = Models([diffuse])

file_model = "diffuse.yaml"
models_diffuse.write(file_model, overwrite=True)

In [ ]:
dataset = MapDataset.read("dataset.fits.gz")
dataset.models.extend(models_diffuse)
print(dataset.models)

In [ ]:
sampler = MapDatasetEventSampler(random_state=0)
events = sampler.run(dataset, observation)

In [ ]:
events.peek()

## Simulate multiple observations

In [ ]:
tmin = np.array([0, 5, 9]) * u.hr
tmax = np.array([2, 6.5, 12]) * u.hr

In [ ]:
for obs_id in np.arange(3):
    observation = Observation.create(
        obs_id=1001, pointing=POINTING, 
        tstart=tmin[obs_id], tstop=tmax[obs_id],
        livetime=tmax[obs_id]-tmin[obs_id], 
        irfs=irfs
    )
    empty = MapDataset.create(WCS_GEOM, energy_axis_true=ENERGY_AXIS_TRUE, migra_axis=MIGRA_AXIS)
    maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])
    dataset = maker.run(empty, observation)
    dataset.write(f'dataset_{obs_id:04d}.fits.gz', overwrite=True)

    dataset.models.extend(models_pntgaus)
    sampler = MapDatasetEventSampler(random_state=0)
    events = sampler.run(dataset, observation)
    events.table.write(f"events_{obs_id:04d}.fits.gz", overwrite=True)

In [ ]:
path = Path("./")
paths = list(path.rglob("events*.fits.gz"))
data_store = DataStore.from_events_files(paths)
observations = data_store.get_observations()

### Exercise
- Change the spatial model and the spectrum of the simulated Sky model;
- Include a temporal model in the simulation